In [22]:
import os
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
# from langchain_ollama.llms import OllamaLLM

from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [33]:
OLLAMA_BASE_URL = "http://host.docker.internal:11434"
OLLAMA_MODEL_ID = "qwen2.5"

In [34]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [35]:
llm = OllamaFunctions(model=OLLAMA_MODEL_ID, base_url=OLLAMA_BASE_URL, verbose=True)

In [36]:
class Tables(BaseModel):
    """Table in SQL database."""

    table_names: list[str] = Field(description="List of names of relevant tables in SQL database.")

table_names = "\n".join(db.get_usable_table_names())
system = f"""Below are the available table names:

{table_names}

Return the names of ALL the SQL tables that MIGHT be relevant to the user question.
Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{input}"),
    ]
)
llm_with_tools = llm.bind_tools([Tables])
output_parser = PydanticToolsParser(tools=[Tables])

table_chain = prompt | llm_with_tools | output_parser

table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[Tables(table_names=['Album', 'Track', 'Genre'])]